In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [8]:
def extract_frame(video_path, frame_number=0):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # print(f"Total frames: {total_frames}")
    # print(f"FPS: {fps}")
    
    # Set frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    
    # Read frame
    ret, frame = cap.read()
    
    if not ret or frame is None:
        print("Error: Could not read frame.")
        return None, total_frames, fps
    
    # cut frame to scan area
    frame = frame[60:520, 300:620]
    # Release video capture object
    cap.release()

    return frame, total_frames, fps

video_path ='/cosma7/data/dp004/rrtx34/ultrasound/JCUH/024/JCUH_024_LUS_3/20240118_142615_0003.AVI'
# Extract single frame (frame 10)
frame, total_frames, fps = extract_frame(video_path, 10)

## checking where to cut frame
# plt.figure(figsize=(10,8))
# plt.imshow(frame)
# plt.grid(True,'both', color='white', linestyle='-', linewidth=0.5, alpha=0.7)
# # Control grid spacing (every N pixels)
# plt.grid(True, which='major', color='white', linewidth=0.5)
# ax = plt.gca()
# ax.set_xticks(np.arange(0, frame.shape[1], 20))  # Every 50 pixels horizontally
# ax.set_yticks(np.arange(0, frame.shape[0], 20))  # Every 50 pixels vertically
# plt.title('Full Frame')
# plt.show()



# count total pixels in cut frame
total_pixels = frame.shape[0] * frame.shape[1]
print("Total pixels in cut frame:", total_pixels)

# find mean pixel value of the frame
mean_pixel_value = np.mean(frame)
# # find variance of pixel values in the frame
variance_pixel_value = np.var(frame)
print("Mean pixel value of the frame:", mean_pixel_value)
print("Variance of pixel values in the frame:", variance_pixel_value)

Total pixels in cut frame: 147200
Mean pixel value of the frame: 37.328978713768116
Variance of pixel values in the frame: 1426.9623640384962


In [9]:
# find the variance in mean pixel value for a video
def calculate_video_stats(video_path):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    mean_values = []
    frame_variance = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # cut frame to scan area
        frame = frame[60:520, 300:620]
        
        mean_pixel_value = np.mean(frame)
        frame_variance_value = np.var(frame)
        mean_values.append(mean_pixel_value)
        frame_variance.append(frame_variance_value)
    
    cap.release()
    
    variance_of_means = np.var(mean_values)
    average_frame_variance = np.mean(frame_variance)
    return variance_of_means, average_frame_variance

video_variance = calculate_video_stats(video_path)
print("Variance of mean pixel values across video frames:", video_variance[0], "Average variance of pixel values within frames:", video_variance[1])

Variance of mean pixel values across video frames: 4.137489093163955 Average variance of pixel values within frames: 1336.451423043533


In [10]:
# compare video stats between duplicate videos
video_3 = calculate_video_stats('/cosma7/data/dp004/rrtx34/ultrasound/JCUH/034/JCUH_034_LUS_4_Supine/20240430_171120_0003.MP4')
video_4 = calculate_video_stats('/cosma7/data/dp004/rrtx34/ultrasound/JCUH/034/JCUH_034_LUS_4_Supine/20240430_171120_0004.MP4')
print("Video 3 - Variance of mean pixel values across video frames:", video_3[0], "Average variance of pixel values within frames:", video_3[1])
print("Video 4 - Variance of mean pixel values across video frames:", video_4[0], "Average variance of pixel values within frames:", video_4[1])

def select_duplicate_videos(vid1_path, vid2_path):
    vid1_stats = calculate_video_stats(vid1_path)
    vid2_stats = calculate_video_stats(vid2_path)

    frame_variance_diff = abs(vid1_stats[1] - vid2_stats[1])
    video_variance_diff = abs(vid1_stats[0] - vid2_stats[0])    
    
    # return video with higher variance in frames
    if frame_variance_diff > video_variance_diff:
        if vid1_stats[1] > vid2_stats[1]:
            return vid1_path
        else:
            return vid2_path
        
    elif video_variance_diff > frame_variance_diff:
        if vid1_stats[0] < vid2_stats[0]:
            return vid1_path
        else:
            return vid2_path


Video 3 - Variance of mean pixel values across video frames: 0.578787625840791 Average variance of pixel values within frames: 171.2326905628656
Video 4 - Variance of mean pixel values across video frames: 0.49148974125709605 Average variance of pixel values within frames: 175.91861746761055


In [11]:
from tqdm.auto import tqdm

df_duplicates = pd.read_csv('LUS_data_duplicate_scan_labels.csv')

# Group by Patient ID, Scan No, and Scan Label to find all duplicate groups
grouped = df_duplicates.groupby(['Patient ID', 'Scan No', 'Scan Label'])

preferences_list = []

# Process each group of duplicates
for group_key, group_df in tqdm(grouped, desc="Processing duplicate groups"):
    # Skip groups with only 1 video (not duplicates)
    if len(group_df) < 2:
        continue
    
    # Get all video paths in this duplicate group
    video_paths = group_df['File Path'].tolist()
    
    # Calculate stats for all videos in the group
    video_stats = {}
    for video_path in video_paths:
        stats = calculate_video_stats(video_path)
        if stats is not None:
            video_stats[video_path] = stats

    # Extract all frame variances and video variances for normalization
    frame_vars = [stats[1] for stats in video_stats.values()]
    video_vars = [stats[0] for stats in video_stats.values()]
    
    # Normalize to 0-1 range
    frame_var_min, frame_var_max = min(frame_vars), max(frame_vars)
    video_var_min, video_var_max = min(video_vars), max(video_vars)
    
    # Avoid division by zero
    frame_var_range = frame_var_max - frame_var_min if frame_var_max != frame_var_min else 1
    video_var_range = video_var_max - video_var_min if video_var_max != video_var_min else 1
    
    # Calculate normalized composite score for each video
    video_scores = {}
    for video_path, stats in video_stats.items():
        # Normalize frame variance (higher is better)
        norm_frame_var = (stats[1] - frame_var_min) / frame_var_range
        
        # Normalize video variance (lower is better, so invert)
        norm_video_var = 1 - (stats[0] - video_var_min) / video_var_range
        
        # Equal weighting (0.5 each)
        composite_score = 0.5 * norm_frame_var + 0.5 * norm_video_var
        video_scores[video_path] = composite_score
    
    # Select the best video from the group
    best_video = max(video_scores.keys(), key=lambda v: video_scores[v])
    
    # Record all videos in this group with the preferred one
    preferences_list.append({
        'Patient ID': group_key[0],
        'Scan No': group_key[1],
        'Scan Label': group_key[2],
        'num_duplicates': len(video_paths),
        'all_videos': video_paths,
        'preferred': best_video
    })

df_preferences = pd.DataFrame(preferences_list)



Processing duplicate groups:   0%|          | 0/164 [00:00<?, ?it/s]

In [12]:
df_preferences.to_csv('LUS_duplicate_video_preferences.csv', index=False)